<a href="https://colab.research.google.com/github/akash-kaul/OneClickDashboardTigerGraphGraphathon/blob/master/covid19starterkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports we need
from ipywidgets import interact, interactive, fixed, interact_manual, widgets, Layout
from IPython.display import display, HTML
from IPython.utils import io
from tqdm import tqdm
from ipywidgets.widgets import VBox, HBox, Box, Label
import time
from glob import glob
import requests
from functools import wraps
import re
import os
import sys
import json

# Stylesheet we use for icons
display(HTML('<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"/>'))

# output field we use to display popup and error messages
out = widgets.Output()

# Establish format for all boxes
form_item_layout = Layout(
    display='inline-flex',
    flex_flow='row',
    justify_content='space-between'
)

# Create a text field for hostname
hostname = widgets.Text(
    value='',
    placeholder='TigerGraph Hostname (ex: https://tg.i.tgcloud.io)',
    disabled=False
)

# Create a text field for username
username = widgets.Text(
    value='tigergraph',
    placeholder='TigerGraph Username',
    disabled=False
)

# Create a text field for password
password = widgets.Password(
    value='',
    palceholder='TigerGraph Password',
    disabled=False,
)

# Create a text field for graphname
graphname = widgets.Text(
    value='MyGraph',
    placeholder='TigerGraph Graph Name',
    disabled=False,
)

# Create a checkbox field for whether a cert is needed or not
cert = widgets.Checkbox(
    value=False,
    disabled=False,
    indent=False
)

# Create initial connect button to connect to server
connect = widgets.Button(
    description='Launch Dashboard',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to connect to graph',
    icon='plug', # (FontAwesome names without the `fa-` prefix)
    layout=Layout(width='100%')
)

# Create label for error message if can't connect to graph
error = widgets.Label(
    value='Something went wrong!! Make sure your server is live and your information is entered correctly',
    disabled=False,
    layout=Layout(width='50%')

)
# Create initial layout for input form
form_items = [
    Box([Label(value='Hostname'), hostname], layout=form_item_layout),
    Box([Label(value='Username'), username], layout=form_item_layout),
    Box([Label(value='Password'), password], layout=form_item_layout),
    Box([Label(value='Graph Name'), graphname], layout=form_item_layout),
    Box([Label(value='Cert Needed'), cert], layout=form_item_layout),
    Box([connect], layout=form_item_layout)
]

# Method that is run when they click the connect button
def connectToGraph(b):
    with tqdm(total=100, leave=False) as pbar:      # Create loading bar
        out.clear_output()
        pbar.bar_format="{percentage:3.0f}%|{bar:10}|{desc}"    # Set bar format

        # Change connect button from 'connect' to 'loading', chnage color from blue to orange
        connect.icon='spinner'
        connect.description='Loading'
        connect.button_style='warning'

        #--pyTigerGraph_BEGIN--#        Import and install pyTigerGraph package (allows us to connect to TG server easily)
        pbar.set_description_str('Installing pyTigerGraph')
        with io.capture_output() as captured:       # Code used to supress output to console (so doesn't show packages being installed)
            !pip install pytigergraph==0.0.7
            import pyTigerGraph as tg
        pbar.update(10)
        #--pyTigerGraph_END--#

        #--packages_BEGIN--#        Install other packages that we will need for the dashboard (like Streamlit)
        pbar.set_description_str('Installing other packages')
        with io.capture_output() as captured: 
            !pip install -q streamlit
            !pip install flat-table
        pbar.update(30)
         #--packages_END--#     

        #--Git_Clone_BEGIN--#       Clone 2 Github Repos containing GSQL query and streamlit script, store names/paths of demos
        gsql_query = ''
        pbar.set_description_str('Grabbing Visualization Code')
        if not os.path.exists('/content/5c5821e4c52a94ac34c601bd05770fb8/covid19starterkit.py'):        
            with io.capture_output() as captured:
                !git clone https://gist.github.com/5c5821e4c52a94ac34c601bd05770fb8.git
        if not os.path.exists('/content/c8209824c5043f34f69b1352195be1c5/streamlit.gsql'):
            with io.capture_output() as captured:
                !git clone https://gist.github.com/rohanshiva/c8209824c5043f34f69b1352195be1c5
        else:
            time.sleep(2)
        os.chdir('/content/5c5821e4c52a94ac34c601bd05770fb8')
        with open('/content/c8209824c5043f34f69b1352195be1c5/streamlit.gsql', 'r') as file:
            gsql_query = file.read()
        pbar.update(20)
        #--Git_Clone_END--#


        # --Establish_Connection_BEGIN--#          Connect to TG server (if error display error message), install GSQL query we cloned
        pbar.set_description_str('Connecting to TigerGraph')
        # Try connecting to graph, if fails stop and print error message
        try:
            with io.capture_output() as captured:
                graph = tg.TigerGraphConnection(host=hostname.value, username=username.value, graphname=graphname.value, password=password.value, useCert=cert.value)
                API_Secret = graph.createSecret()
                API_Token = graph.getToken(API_Secret, setToken=True, lifetime=None)[0]
        except Exception as e:
            print(e)
            pbar.close()
            out.clear_output()
            with out:
                display(error)
                connect.icon='plug'
                connect.description='Connect to TigerGraph'
                connect.button_style='info'
            return
        pbar.update(10)
        pbar.set_description_str('Uploading Streamlit Query (Takes ~1 min)')
        with io.capture_output() as captured:
            graph.gsql('drop query streamlit')
            gsql_query += 'INSTALL QUERY streamlit'
            graph.gsql(gsql_query)
        pbar.update(20)
        #--Establish_Connection_END--# 

        #--deploy_BEGIN--#      Configure ngrok server, ensure tunnel is created (while loop)
        pbar.set_description_str('Configuring Server')
        ngrok_url = 'You can access the Dashboard at: '
        with io.capture_output() as output_url:
            try:
                !wget -c https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
                !unzip -o ngrok-stable-linux-amd64.zip
                get_ipython().system_raw('./ngrok http 8501 &')
            except Exception as e:
                print(e)
        while(True):
            with io.capture_output() as output_url:
                !curl -s http://localhost:4040/api/tunnels
            x = output_url.stdout
            x = json.loads(x)
            if x['tunnels']:
                ngrok_url += x['tunnels'][0]['public_url']
                break
        pbar.set_description_str('Launching App')
        time.sleep(1)
        pbar.update(10)
        #--deploy_END--#

        #--Wrap_Up_BEGIN--#     Close loading bar, disable all text boxes in the form, update button to 'success' with green color, and launch app
        connect.icon='check-circle'
        connect.description='Success'
        connect.button_style='success'
        time.sleep(1)
        pbar.close()
        username.disabled=True
        password.disabled=True
        hostname.disabled=True
        cert.disabled=True
        connect.disabled=True
        graphname.disabled=True
        # Display url so user can access it
        print(ngrok_url)
        # Write user data (hostname, graphname, etc.) to text file that streamlit script then reads in
        f = open("streamlit.txt", "w")
        graph_info = [hostname.value, username.value, password.value, graphname.value, str(cert.value)]
        f.write(" ".join(graph_info))
        f.close()
        # Run streamlit
        with io.capture_output() as captured:
            !streamlit run covid19starterkit.py
        #--Wrap_Up_END--#

# Create the initial form (asks for hostname, username, etc.)
connect.on_click(connectToGraph)
form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='30%'
))

# Display the form
display(form, out)

Box(children=(Box(children=(Label(value='Hostname'), Text(value='', placeholder='TigerGraph Hostname (ex: http…

Output()

You can access the Dashboard at: https://762dc72c6dbd.ngrok.io
